In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Butler

Load data for all of the visits included in DP1

In [ ]:
from lsst.daf.butler import Butler

repo="/repo/main"
collection="LSSTComCam/runs/DRP/DP1/w_2025_10/DM-49359"
butler = Butler(repo, collections=collection)

In [ ]:
registry = butler.registry

In [ ]:
"""
# Determine which dataset types exist in the collection
for datasetType in registry.queryDatasetTypes():
    if registry.queryDatasets(datasetType, collections=collection).any(execute=False, exact=False):
        # Limit search results to the data products
        if ('_config' not in datasetType.name) and ('_log' not in datasetType.name) and ('_metadata' not in datasetType.name) and ('_resource_usage' not in datasetType.name):
            print(datasetType)
"""

In [ ]:
refs = list(butler.registry.queryDatasets('finalVisitSummary'))

In [ ]:
len(refs)

In [ ]:
refs = list(butler.registry.queryDatasets('visitTable'))

In [ ]:
len(refs)

In [ ]:
visitTable = butler.get(refs[0])

In [ ]:
visitTable.columns

In [ ]:
visitTable['obsStart']

In [ ]:
visitTable['visitId']

# Consolidated Database

Not strictly necessary, but a consistency check for the visits that make it into DP1

In [ ]:
import os
os.environ["no_proxy"] += ",.consdb"

from lsst.summit.utils import ConsDbClient

client = ConsDbClient("http://consdb-pq.consdb:8080/consdb")

In [ ]:
instrument = 'lsstcomcam'

In [ ]:
visits_query = f'''
    SELECT 
        * 
    FROM 
        cdb_{instrument}.visit1 
    WHERE 
        science_program in ('BLOCK-320', 'PP-SURVEY', 'BLOCK-T248', 'BLOCK-T277')
'''

visits = client.query(visits_query).to_pandas()

In [ ]:
len(visits)

In [ ]:
visits['visit_id']

# Analysis

Unfortunately, the metadata conventions for target names evolved during the ComCam on-sky campaign. As a first step, use a spatial match to associate visits with targets.

In [ ]:
bands = {'u': '#0c71ff', 'g': '#49be61', 'r': '#c61c00', 'i': '#ffc200', 'z': '#f341a2', 'y': '#5d0000'}

In [ ]:
plt.figure()
plt.scatter(visitTable['ra'], visitTable['dec'])
plt.xlabel('RA (deg)')
plt.ylabel('dec (deg)')

In [ ]:
len(np.intersect1d(visits['visit_id'], visitTable['visitId']))

In [ ]:
assert len(np.intersect1d(visits['visit_id'], visitTable['visitId'])) == len(visitTable)

In [ ]:
visits_coord = SkyCoord(visitTable['ra'], visitTable['dec'], unit='deg', frame='icrs')

In [ ]:
from astropy.coordinates import SkyCoord
from astroplan import FixedTarget

targets = [
    FixedTarget(SkyCoord(6.022329,-72.081444, unit='deg', frame='icrs'), name='47_Tuc'),
    FixedTarget(SkyCoord(37.9, 7.0, unit='deg', frame='icrs'), name='Rubin_SV_38_7'),
    FixedTarget(SkyCoord(39.9971, -34.4492, unit='deg', frame='icrs'), name='Fornax_dSph'),
    FixedTarget(SkyCoord(53.125, -28.1, unit='deg', frame='icrs'), name='ECDFS'),
    FixedTarget(SkyCoord(59.1004, -48.73, unit='deg', frame='icrs'), name='EDFS_ComCam'),
    FixedTarget(SkyCoord(95.0, -25.0, unit='deg', frame='icrs'), name='Rubin_SV_095_-25'),
    FixedTarget(SkyCoord(106.23, -10.51, unit='deg', frame='icrs'), name='Seagull'),
]

In [ ]:
target_column = np.empty(len(visitTable), dtype="S20")

for target in targets:
    angsep = target.coord.separation(visits_coord).deg
    selection_angle = angsep < 2.

    target_column[selection_angle] = target.name

In [ ]:
np.unique(target_column)

In [ ]:
target_column.astype(str)

In [ ]:
visitTable['target'] = target_column.astype(str)

In [ ]:
visitTable['target'].value_counts()

In [ ]:
visitTable

# Plots

In [ ]:
plt.figure(figsize=(12, 5))

for ii, target in enumerate(targets):
    selection_target = (visitTable['target'] == target.name)
    
    for jj, band in enumerate(bands):

        if ii == 0:
            label = band
        else:
            label = None
        
        selection_band = (visitTable['band'] == band)
        selection = (selection_target & selection_band)
        y = np.tile(ii, np.sum(selection)) - (jj * 0.12) + 0.3
        plt.scatter(visitTable['expMidptMJD'][selection], y, c=bands[band], marker='o', label=label)

plt.yticks(np.arange(len(targets)), [t.name for t in targets])
plt.xlabel('MJD')

plt.grid(axis='x', c='0.8', lw=1)
plt.gca().set_axisbelow(True)

for y in (np.arange(0, len(targets)) + 0.5):
    plt.axhline(y, c='0.8', lw=1)

plt.ylim(-0.5, len(targets) - 0.5)

plt.legend(loc='center left', bbox_to_anchor=(1., 0.5))

In [ ]:
#plt.figure()
#plt.hist(visitTable['expMidptMJD'], bins=50) #, cumulative='true')

In [ ]:
plt.figure()
for band in bands:
    selection_band = (visitTable['band'] == band)
    plt.scatter(visitTable['expMidptMJD'][selection_band], visitTable['expMidptMJD'][selection_band] % 1, marker='_', c=bands[band], label=band)

plt.legend(loc='center left', bbox_to_anchor=(1., 0.5))

plt.xlabel('MJD')
plt.ylabel('Fractional Day')